In [179]:
import time
import nltk
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [67]:
train=pd.read_csv("RCV1.txt", delim_whitespace=True, skipinitialspace=True, names= ['Word','POStag','CHUNKtag','NEtag'])
testa=pd.read_csv("testa.txt", delim_whitespace=True, skipinitialspace=True, names= ['Word','POStag','CHUNKtag','NEtag'])
testb=pd.read_csv("testb.txt", delim_whitespace=True, skipinitialspace=True, names= ['Word','POStag','CHUNKtag','NEtag'])

In [68]:
print(train.shape,testa.shape,testb.shape)

(204567, 4) (51578, 4) (46666, 4)


In [69]:
train['CHUNKtag'].describe()
train = train.dropna()

In [70]:
train.shape

(202386, 4)

In [137]:
testa.head()

,Word,POStag,CHUNKtag,NEtag
0,-DOCSTART-,-X-,-X-,O
1,CRICKET,NNP,I-NP,O
2,-,:,O,O
3,LEICESTERSHIRE,NNP,I-NP,I-ORG
4,TAKE,NNP,I-NP,O


# Preprocess train

In [71]:
sparse = pd.get_dummies(train['NEtag'])
sparse.drop(sparse.columns[7], axis=1, inplace=True)
train['ORG'] = sparse['I-ORG'] + sparse['B-ORG']
train['LOC'] = sparse['I-LOC'] + sparse['B-LOC']
train['MISC'] = sparse['I-MISC'] + sparse['B-MISC']
train['PER'] = sparse['I-PER']

In [72]:
gazloc = nltk.corpus.gazetteers.words(fileids=['countries.txt','uscities.txt','usstates.txt','usstateabbrev.txt','mexstates.txt','caprovinces.txt'])
gazper = nltk.corpus.names.words(fileids=['male.txt','female.txt'])
gazmisc = nltk.corpus.gazetteers.words(fileids=['nationalities.txt'])

In [73]:
def gazetteer( text , ref ):
    res = []
    for word in text:
        if word in ref :
            res.append(1)
        else :
            res.append(0)
    return res


In [74]:
result = gazetteer(train['Word'] , gazloc)
x = train['LOC'] - result
print ('Real is ',sum(train['LOC']),' and we found ',sum(x),' locations')

Real is  8297  and we found  4445  locations


In [75]:
result = gazetteer(train['Word'] , gazper)
x = train['PER'] - result
print ('Real is ',sum(train['PER']),' and we found ',sum(x), ' persons')

Real is  11128  and we found  5365  persons


In [76]:
result = gazetteer(train['Word'] , gazmisc)
x = train['MISC'] - result
print ('Real is ',sum(train['MISC']),' and we found ',sum(x), ' miscellaneous')

Real is  4593  and we found  3173  miscellaneous


Format BIO:

Before
Inside
Outside

In [77]:
def capitalize(text):
    cap = []
    for x in text:
        if x[0].isupper():
            cap.append(1)
        else : 
            cap.append(0)
    return cap

#capitalize(['.','EU','Germany','represents'])

In [78]:
def fullcap(text):
    cap = []
    for x in text:
        if x.isupper():
            cap.append(1)
        else : 
            cap.append(0)
    return cap

In [79]:
def length(text):
    length = []
    for x in text:
        length.append(len(x))
    return length

In [80]:
train['leng'] = length(train['Word']) 

In [81]:
train['Cap']=capitalize(train['Word'])

In [82]:
train['Cap2']=fullcap(train['Word'])

In [83]:
POStag = train['POStag'].tolist()
BPOStag = train['POStag'].tolist()
OPOStag = train['POStag'].tolist()

m = len(train['Word'])-1
for i in range(1,m):
    BPOStag[i] = POStag[i-1]
    OPOStag[i] = POStag[i+1]
    
train['BPOStag'] = BPOStag
train['OPOStag'] = OPOStag

In [84]:
train['GAZMISC'] = gazetteer(train['Word'] , gazmisc)
train['GAZPER'] = gazetteer(train['Word'] , gazper)
train['GAZLOC'] = gazetteer(train['Word'] , gazloc)

In [85]:
train.head()

,Word,POStag,CHUNKtag,NEtag,ORG,LOC,MISC,PER,leng,Cap,Cap2,BPOStag,OPOStag,GAZMISC,GAZPER,GAZLOC
0,-DOCSTART-,-X-,-X-,O,0,0,0,0,10,0,1,-X-,-X-,0,0,0
1,EU,NNP,I-NP,I-ORG,1,0,0,0,2,1,1,-X-,VBZ,0,0,0
2,rejects,VBZ,I-VP,O,0,0,0,0,7,0,0,NNP,JJ,0,0,0
3,German,JJ,I-NP,I-MISC,0,0,1,0,6,1,0,VBZ,NN,1,0,0
4,call,NN,I-NP,O,0,0,0,0,4,0,0,JJ,TO,0,0,0


# Preprocess test

In [141]:
word = []
for x in testa['Word']:
    word.append(str(x))

testa['Word']=word

In [142]:
preprocess(testa)

In [170]:
POStag = testa['POStag'].tolist()

for index, item in enumerate(POStag):
    if item == 'O' :
        POStag[index] = '-X-'
        
testa['POStag'] = POStag

In [171]:
POStag = testa['POStag'].tolist()
BPOStag = testa['POStag'].tolist()
OPOStag = testa['POStag'].tolist()

m = len(testa['Word'])-1
for i in range(1,m):
    BPOStag[i] = POStag[i-1]
    OPOStag[i] = POStag[i+1]
    
testa['BPOStag'] = BPOStag
testa['OPOStag'] = OPOStag

In [172]:
sparse = pd.get_dummies(testa['NEtag'])
sparse.head()

,B-MISC,I-LOC,I-MISC,I-ORG,I-PER,O
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,1,0,0
4,0,0,0,0,0,1


In [173]:
sparse = pd.get_dummies(testa['NEtag'])
sparse.drop(sparse.columns[5], axis=1, inplace=True)
testa['ORG'] = sparse['I-ORG'] 
testa['LOC'] = sparse['I-LOC'] 
testa['MISC'] = sparse['I-MISC'] + sparse['B-MISC']
testa['PER'] = sparse['I-PER']

In [174]:
testa.head()

,Word,POStag,CHUNKtag,NEtag,Cap,Cap2,leng,GAZMISC,GAZPER,GAZLOC,BPOStag,OPOStag,ORG,LOC,MISC,PER
0,-DOCSTART-,-X-,-X-,O,0,1,10,0,0,0,-X-,-X-,0,0,0,0
1,CRICKET,NNP,I-NP,O,1,1,7,0,0,0,-X-,:,0,0,0,0
2,-,:,O,O,0,0,1,0,0,0,NNP,NNP,0,0,0,0
3,LEICESTERSHIRE,NNP,I-NP,I-ORG,1,1,14,0,0,0,:,NNP,1,0,0,0
4,TAKE,NNP,I-NP,O,1,1,4,0,0,0,NNP,IN,0,0,0,0


In [175]:
datatest = testa.drop(['Word','NEtag','ORG','LOC','MISC','PER'], axis=1)
sparse = pd.get_dummies(datatest.iloc[:,[0,1,8,9]])
datatest.drop(['POStag','OPOStag','BPOStag','CHUNKtag'] , axis=1 , inplace=True)
datatest = pd.concat([datatest , sparse] , axis=1)

In [176]:
targettest = testa.iloc[:,12:16]

# Prédiction

In [131]:
data = train.drop(['Word','NEtag','ORG','LOC','MISC','PER'], axis=1)
sparse = pd.get_dummies(data.iloc[:,[0,1,5,6]])
data.drop(['POStag','OPOStag','BPOStag','CHUNKtag'] , axis=1 , inplace=True)
data = pd.concat([data , sparse] , axis=1)

In [132]:
target = train.iloc[:,4:8]

In [133]:
target.head()

,ORG,LOC,MISC,PER
0,0,0,0,0
1,1,0,0,0
2,0,0,0,0
3,0,0,1,0
4,0,0,0,0


In [180]:
start = time.time()
model = RandomForestClassifier()
model.fit(data , target)
end = time.time()
print(model.score(data , target))
predict = model.predict(datatest)
print(metrics.accuracy_score(targettest, predict))
print ('time :', end - start)

0.945954759717
0.836713327388
time : 7.388756036758423


In [183]:
print(metrics.classification_report(targettest , predict))

             precision    recall  f1-score   support

          0       0.04      0.35      0.07       233
          1       0.02      0.98      0.04        48
          2       0.02      0.91      0.03        23
          3       0.01      0.98      0.03        42

avg / total       0.03      0.55      0.06       346



In [1]:
print(sum(predict))
print(sum(targettest['ORG']), sum(targettest['LOC']), sum(targettest['MISC']), sum(['PER']))

NameError: name 'predict' is not defined

Gazetteer ConLL
Passer le truc en script
Fabriquer des gazetteers dans d'autres langues

In [138]:
def capitalize(text):
    cap = []
    for x in text:
        if x[0].isupper():
            cap.append(1)
        else : 
            cap.append(0)
    return cap

def fullcap(text):
    cap = []
    for x in text:
        if x.isupper():
            cap.append(1)
        else : 
            cap.append(0)
    return cap

def length(text):
    length = []
    for x in text:
        length.append(len(x))
    return length

def gazetteer( text , ref ):
    res = []
    for word in text:
        if word in ref :
            res.append(1)
        else :
            res.append(0)
    return res

def preprocess(data):
    data['Cap']=capitalize(data['Word'])
    data['Cap2']=fullcap(data['Word'])
    data['leng'] = length(data['Word'])
    data['GAZMISC'] = gazetteer(data['Word'] , gazmisc)
    data['GAZPER'] = gazetteer(data['Word'] , gazper)
    data['GAZLOC'] = gazetteer(data['Word'] , gazloc) 

In [ ]:
start = time.time()
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='saga').fit(words, NE)
print(mul_lr.score(words , sparse))
end = time.time()
print ('time :', end - start)